## **1. Работа с базой данных из Python**

>✍ Ранее в курсе мы познакомились с основами SQL (язык запросов к базам данных) и Python (язык программирования). Возможно, у вас возникал вопрос: можно ли объединить два этих механизма? Можно ли работать с базами данных и выполнять SQL-запросы средствами Python? И если можно, то как это сделать?

>В курсе для удобства работы с данными мы использовали Metabase. Однако Metabase — это не сама база данных, а [BI-инструмент](https://proglib.io/p/chto-takoe-bi-sistemy-i-zachem-oni-nuzhny-2021-05-10), то есть удобный интерфейс для взаимодействия с данными, которые хранятся в БД, построения отчётов и графиков. Всё это время наши данные хранились под управлением СУБД [PostgreSQL](https://blog.skillfactory.ru/glossary/postgresql/), а обращались мы к ним через запросы в Metabase.

В реальной деятельности дата-сайентисту требуется не только писать запрос к данным, но и затем обрабатывать его результаты с помощью Python. Для решения таких задач требуется некоторое средство, которое будет связывать Python и PostgreSQL так, чтобы мы могли с помощью Python отправлять запросы в Postgres и принимать оттуда результаты.

Именно таким средством является пакет [psycopg2](https://www.psycopg.org/docs/), о котором мы сейчас и поговорим.

### **УСТАНОВКА PSYCOPG2**

Установить данный пакет на любой платформе можно одним способом — с помощью pip.

Запустите в отдельной ячейке Jupyter Notebook следующий код:


In [5]:
pip install psycopg2

  Using cached psycopg2-2.9.5.tar.gz (384 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [25 lines of output]
      /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/setuptools/config/setupcfg.py:463: SetuptoolsDeprecationWarning: The license_file parameter is deprecated, use license_files instead.
        warnings.warn(msg, warning_class)
      running egg_info
      creating /private/var/folders/6c/s0qxhsyd00x3nt57hw3r4w9c0000gp/T/pip-pip-egg-info-90u43cc6/psycopg2.egg-info
      writing /private/var/folders/6c/s0qxhsyd00x3nt57hw3r4w9c0000gp/T/pip-pip-egg-info-90u43cc6/psycopg2.egg-info/PKG-INFO
      writing dependency_links to /private/var/folders/6c/s0qxhsyd00x3nt57hw3r4w9c0000gp/T/pip-pip-egg-info-90u43cc6/psycopg2.egg-info/dependency_links.txt
      writing top-level names to /private/var/folders/6c/s0qxhsyd00x3nt57hw3r4w9c0000gp/T/p

Если при установке возникает ошибка, похожая на “connection error: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:598)”, попробуйте установить пакет следующим образом:

In [3]:
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org --trusted-host pypi.python.org psycopg2

  Using cached psycopg2-2.9.5.tar.gz (384 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [25 lines of output]
      /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/setuptools/config/setupcfg.py:463: SetuptoolsDeprecationWarning: The license_file parameter is deprecated, use license_files instead.
        warnings.warn(msg, warning_class)
      running egg_info
      creating /private/var/folders/6c/s0qxhsyd00x3nt57hw3r4w9c0000gp/T/pip-pip-egg-info-8fqcom1m/psycopg2.egg-info
      writing /private/var/folders/6c/s0qxhsyd00x3nt57hw3r4w9c0000gp/T/pip-pip-egg-info-8fqcom1m/psycopg2.egg-info/PKG-INFO
      writing dependency_links to /private/var/folders/6c/s0qxhsyd00x3nt57hw3r4w9c0000gp/T/pip-pip-egg-info-8fqcom1m/psycopg2.egg-info/dependency_links.txt
      writing top-level names to /private/var/folders/6c/s0qxhsyd00x3nt57hw3r4w9c0000gp/T/p

Если всё прошло успешно, вы увидите сообщение, подобное этому:

Collecting psycopg2

Downloading psycopg2-2.7.5

Installing collected packages: psycopg2

Successfully installed psycopg2-2.7.5
Цифры могут отличаться, так как пакеты развиваются и номер их версии увеличивается.

Теперь посмотрим, как можно подключиться к нашей базе и выполнять запросы из Jupyter-ноутбука.

Для подключения нам потребуются следующие данные:

dbname — название базы, к которой нужно подключиться;
user — имя пользователя в СУБД;
password — пароль;
host — адрес, по которому нужно подключиться;
port — порт, к которому нужно подключиться (по умолчанию равен 5432).
Для нашей базы эти параметры такие:

DBNAME = 'skillfactory'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

Ноутбук из скринкаста.

НАСТРОЙКИ

Начинаем с импорта библиотек:

import psycopg2
import pandas as pd
Создаём соединение с заданными параметрами:

connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)
Запрос:

# код запроса представляет из себя строковую переменную
n = 10
query = f'''select *
           from sql.pokemon
           limit {n}
        '''
# лучше использовать f-строки — так удобнее передавать параметры
ВЫПОЛНЕНИЕ ЗАПРОСА

# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
df = pd.read_sql_query(query, connection)
df
ФИНАЛ

# не забываем закрыть соединение после окончания работы
connection.close()